In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import descarteslabs as dl
import geopandas as gpd
from tqdm.notebook import tqdm

from scripts import deploy_candidate_detect, candidate_detect

In [ ]:
#roi = 'central_america'
countries = ['Portugal', 'Spain', 'Gibraltar', 'France', 'Monaco', 'Italy', 'san_marino', 'malta', 'slovenia', 'croatia', 'macedonia', 'montenegro', 'greece', 'iraq', 'syria', 'lebanon', 'jordan', 'israel']
countries = ['jordan', 'israel']
for roi in countries:
    for dates in [['2021-01-01', '2022-07-01']]:
        start_date = dates[0]
        end_date = dates[1]
        model_name = '0.0.11'
        product_name = f'{roi}_v{model_name}_{start_date}_{end_date}'
        pred_threshold = 0.6
        min_sigma = 5.0
        band = 'median'
        product_id = f"{product_name}_blobs_thresh_{pred_threshold}_min-sigma_{min_sigma}_area-thresh_0.0025_band-{band}"

        run_local = False

        args = [
            '--product_name',
            product_name,
            '--pred_threshold',
            str(pred_threshold),
            '--min_sigma',
            str(min_sigma),
            '--band',
            band,
        ]
        if run_local:
            args.append('--run_local')
        
        # Because of the way DL uploads modules when queuing async tasks, we need to launch from the scripts/ folder
        %cd ../scripts
        %pwd

        deploy_candidate_detect.main(args)

### Download data after the task is complete

In [ ]:
countries = ['Portugal', 'Spain', 'Gibraltar', 'France', 'Monaco', 'Italy', 'san_marino', 'malta', 'slovenia', 'croatia', 'macedonia', 'montenegro', 'greece', 'iraq', 'syria', 'lebanon', 'jordan', 'israel']
for roi in countries:
    for dates in [['2021-01-01', '2022-07-01']]:
        start_date = dates[0]
        end_date = dates[1]
        model_name = '0.0.11'
        product_name = f'{roi}_v{model_name}_{start_date}_{end_date}'
        pred_threshold = 0.6
        min_sigma = 5.0
        band = 'median'
        product_id = f"{product_name}_blobs_thresh_{pred_threshold}_min-sigma_{min_sigma}_area-thresh_0.0025_band-{band}"
        fc_id = [fc.id for fc in dl.vectors.FeatureCollection.list() if product_id in fc.id][0]
        fc = dl.vectors.FeatureCollection(fc_id)

        roi_file = f'../data/boundaries/{roi}.geojson'
        region = gpd.read_file(roi_file)['geometry']
        features = []
        for i, r in enumerate(region):
            print("region", i)
            candidates = fc.filter(r).features()
            for elem in tqdm(candidates):
                features.append(elem.geojson)
        coords = [feat['geometry']['coordinates'] for feat in features]
        if len(coords) > 0:
            merged = candidate_detect.merge_similar_sites(coords, search_radius=0.01, plot=True)
            directory = f"../data/model_outputs/candidate_sites/{model_name}"
            merged.to_file(os.path.join(directory, product_id.split(':')[-1] + '.geojson'))
        else:
            print("No candidates found in", roi)

In [ ]:
coords = [feat['geometry']['coordinates'] for feat in features]
merged = candidate_detect.merge_similar_sites(coords, search_radius=0.01, plot=True)
directory = f"../data/model_outputs/candidate_sites/{model_name}"
merged.to_file(os.path.join(directory, product_id.split(':')[-1] + '.geojson'))